<a href="https://colab.research.google.com/github/pszemraj/ml4hc-s22-project01/blob/add-gluon-autoML/notebooks/colab/autogluon_tabular_ptbdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> autogluon for ptbdb dataset <center>

- see how autoML does on ptbdb dataset
- docs can be found [here](https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html)


In [1]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

In [2]:
#@title print out GPU info
#@markdown this is the Colab-allocated GPU. If the output here says it fails, no
#@markdown GPU is being used. go to runtime at the top of your colab to set runtime to GPU.


!nvidia-smi

Sun Mar 27 03:48:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install autogluon -q 

In [4]:
#@title define source data parameters

#@markdown - these can also be loaded from gdrive, but I am lazy and `wget` does not require login

ptbdb_train_url = "https://www.dropbox.com/s/nxwz0rlyckcfq0v/torchfmt_ptbdb_train.csv?dl=1" #@param {type:"string"}
ptbdb_train_filename = "ptbdb_train.csv" #@param {type:"string"}
ptbdb_test_url = "https://www.dropbox.com/s/ysey3qdpvgqvoz2/torchfmt_ptbdb_test.csv?dl=1" #@param {type:"string"}
ptbdb_test_filename = "ptbdb_test.csv" #@param {type:"string"}

In [5]:
#@title autogluon parameters 

#@markdown - `max_time_fitting` is in seconds

max_time_fitting =  10800#@param {type:"integer"}
fit_preset = "best_quality" #@param ["best_quality", "high_quality"]
metric = 'roc_auc'  #@param ["roc_auc", "accuracy", "f1"]



In [6]:

!wget $ptbdb_train_url -O $ptbdb_train_filename -q
!wget $ptbdb_test_url -O $ptbdb_test_filename -q

In [7]:
import pandas as pd
example_df = pd.read_csv(ptbdb_train_filename)
data_cols = list(example_df.columns)
_target = data_cols[-1]
data_cols.pop()
_predictors = data_cols # all other columns are numerical predictors

print(f"the target colname is {_target} and\nthe predictor colnames 5 of {len(_predictors)} are {_predictors[:5]}")

the target colname is class_label and
the predictor colnames 5 of 187 are ['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4']


In [8]:
import torch
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset(ptbdb_train_filename)
test_data = TabularDataset(ptbdb_test_filename)
predictor = TabularPredictor(
                            label=_target,
                             eval_metric=metric,
                             ).fit(
                                    train_data, 
                                    time_limit=max_time_fitting,
                                    presets=fit_preset,
                                   ag_args_fit={'num_gpus': torch.cuda.device_count()}
                                   )  


No path specified. Models will be saved in: "AutogluonModels/ag-20220327_034909/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20220327_034909/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    11642
Train Data Columns: 187
Label Column: class_label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['abnormal', 'normal']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = normal, class 0 = abnormal
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (normal) vs negative (abnormal) class.
	To explicitl

In [9]:
results = predictor.fit_summary(show_plot=True)


*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.998942      30.880898  2534.601359                0.004535           5.286282            3       True         26
1           CatBoost_BAG_L2   0.998862      26.144510  2021.160981                0.155204         107.089766            2       True         19
2         LightGBMXT_BAG_L2   0.998651      26.168278  1975.468677                0.178972          61.397462            2       True         15
3       WeightedEnsemble_L2   0.998557       8.626633  1132.504390                0.004494           6.025515            2       True         14
4   RandomForestEntr_BAG_L2   0.998553      26.857858  1931.383121                0.868551          17.311906            2       True         18
5     ExtraTreesGini_BAG_L2   0.998430      26.867577  1916.944789  

In [11]:
extra_metrics=['accuracy', 'roc_auc', 'log_loss', 'f1']

leaderboard = predictor.leaderboard(test_data, extra_metrics=extra_metrics)

leaderboard

/usr/local/lib/python3.7/dist-packages/autogluon/core/metrics/__init__.py:486: RuntimeWarning: divide by zero encountered in log
  return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
/usr/local/lib/python3.7/dist-packages/autogluon/core/metrics/__init__.py:486: RuntimeWarning: divide by zero encountered in log
  return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
/usr/local/lib/python3.7/dist-packages/autogluon/core/metrics/__init__.py:486: RuntimeWarning: divide by zero encountered in log
  return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
/usr/local/lib/python3.7/dist-packages/autogluon/core/metrics/__init__.py:486: RuntimeWarning: divide by zero encountered in log
  return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
/usr/local/lib/python3.7/dist-packages/autogluon/core/metrics/__init__.py:486: RuntimeWarning: divide by zero encountered in log
  return - (y_true * np.log(y_pred)

                      model  score_test  accuracy   roc_auc  log_loss        f1  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   RandomForestGini_BAG_L1    1.000000  1.000000  1.000000 -0.039983  1.000000   0.995136        0.234550       0.823373    16.136983                 0.234550                0.823373          16.136983            1       True          5
1   RandomForestEntr_BAG_L1    1.000000  1.000000  1.000000 -0.038136  1.000000   0.995818        0.237859       0.799363    26.823759                 0.237859                0.799363          26.823759            1       True          6
2           CatBoost_BAG_L1    1.000000  1.000000  1.000000 -0.021306  1.000000   0.996732        0.437297       0.266685   753.443222                 0.437297                0.266685         753.443222            1       True          7
3     ExtraTreesGini_BAG_L1    1.000000  1.00000

,model,score_test,accuracy,roc_auc,log_loss,f1,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini_BAG_L1,1.000000,1.000000,1.000000,-0.039983,1.000000,0.995136,0.234550,0.823373,16.136983,0.234550,0.823373,16.136983,1,True,5
1,RandomForestEntr_BAG_L1,1.000000,1.000000,1.000000,-0.038136,1.000000,0.995818,0.237859,0.799363,26.823759,0.237859,0.799363,26.823759,1,True,6
2,CatBoost_BAG_L1,1.000000,1.000000,1.000000,-0.021306,1.000000,0.996732,0.437297,0.266685,753.443222,0.437297,0.266685,753.443222,1,True,7
3,ExtraTreesGini_BAG_L1,1.000000,1.000000,1.000000,-0.043737,1.000000,0.995754,0.486046,0.829825,3.495134,0.486046,0.829825,3.495134,1,True,8
4,ExtraTreesEntr_BAG_L1,1.000000,1.000000,1.000000,-0.041977,1.000000,0.996033,0.646745,0.878775,3.382455,0.646745,0.878775,3.382455,1,True,9
5,XGBoost_BAG_L1,1.000000,1.000000,1.000000,-0.004847,1.000000,0.997166,0.819513,0.725899,40.844169,0.819513,0.725899,40.844169,1,True,11
6,LightGBM_BAG_L1,1.000000,1.000000,1.000000,-0.003575,1.000000,0.997358,1.046307,1.136761,184.135330,1.046307,1.136761,184.135330,1,True,4
7,LightGBMLarge_BAG_L1,1.000000,1.000000,1.000000,-0.003520,1.000000,0.997005,1.063834,0.797598,308.626297,1.063834,0.797598,308.626297,1,True,13
8,NeuralNetFastAI_BAG_L1,1.000000,1.000000,1.000000,-0.004579,1.000000,0.996750,1.192670,1.235694,126.482172,1.192670,1.235694,126.482172,1,True,10
9,KNeighborsDist_BAG_L1,1.000000,1.000000,1.000000,0.000000,1.000000,0.979353,1.791711,6.530455,0.191176,1.791711,6.530455,0.191176,1,True,2


In [14]:
!pip install openpyxl -q

In [15]:
from google.colab import files
import pandas as pd
from pathlib import Path

_cwd = Path.cwd()

csv_path = _cwd / "ptbdb_autogluon_results.csv"
xlsx_path = _cwd / "ptbdb_autogluon_results.xlsx"

leaderboard.to_csv(csv_path)
leaderboard.to_excel(xlsx_path)


In [16]:
from google.colab import files
import time
files.download(csv_path)
time.sleep(3)
files.download(xlsx_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>